# Case Study Data Quality Analyst

In [ ]:
!pip install plotly
!pip install python_pptx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 10.2 MB/s eta 0:00:00


The data was loaded using the pandas and the purchase exit survey, airings and lookup were loaded into three seperate data frames.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from pptx import Presentation
from pptx.util import Inches, Pt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Load the Excel file
excel_file = 'Analyst_Dataset.xlsx'

# Read each sheet into a separate DataFrame
purchase_survey_df = pd.read_excel(excel_file, sheet_name='Purchase Exit Survey Data')
airings_df = pd.read_excel(excel_file, sheet_name='Airings')
lookup_df = pd.read_excel(excel_file, sheet_name='Lookup')

# Data Preprocessing:

The purchase data was formatted with the correct column headings and the dates which were intially object and integer types were formatted to Y-m-d.

Another issue found was that the data was missing on the dates 9, 20 of september and the 6th of october so these dates were skipped and the issue was noted.

In [ ]:
# Drop irrelevant rows (first three rows) and set the correct header row
purchase_survey_cleaned = purchase_survey_df.drop([0, 1, 2]).reset_index(drop=True)
purchase_survey_cleaned.columns = purchase_survey_cleaned.iloc[0]
purchase_survey_cleaned = purchase_survey_cleaned.drop(0).reset_index(drop=True)

# Drop the 'Source Category' column and any completely empty columns or rows
purchase_survey_cleaned = purchase_survey_cleaned.drop(columns=['Source Category'])
purchase_survey_cleaned = purchase_survey_cleaned.dropna(how='all', axis=1).dropna(how='all', axis=0)

# Function to generate columns for dates, skipping specified dates
def generate_date_columns(start_date, end_date, skip_dates=[]):
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    all_dates = [start + timedelta(days=i) for i in range((end - start).days + 1)]
    return [date.strftime("%Y-%m-%d") for date in all_dates if date.day not in skip_dates]

# Generate dates for September and October
dates_september = generate_date_columns("2017-09-02", "2017-09-30", skip_dates=[1, 9, 20])
dates_october = generate_date_columns("2017-10-01", "2017-10-30", skip_dates=[6])
new_columns = ['Source'] + dates_september + dates_october

# Ensure the length matches the number of columns in your DataFrame
if len(new_columns) == len(purchase_survey_cleaned.columns):
    purchase_survey_cleaned.columns = new_columns
else:
    raise ValueError(f"Length mismatch: Expected {len(purchase_survey_cleaned.columns)} columns, but generated {len(new_columns)} columns.")

The data was reshaped from a wide format to a long format and the dates were converted to datetime objects for easier manipulation. Missing values in the TV Attributable Purchases column are replaced with 0s.

In [ ]:
# Melt the purchase survey data to long format
purchase_survey_long = purchase_survey_cleaned.melt(id_vars=['Source'], var_name='Date', value_name='TV Attributable Purchases')
purchase_survey_long['Date'] = pd.to_datetime(purchase_survey_long['Date'])
purchase_survey_long['TV Attributable Purchases'] = purchase_survey_long['TV Attributable Purchases'].fillna(0)


<ipython-input-4-a5f5c6c3da0a>:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  purchase_survey_long['TV Attributable Purchases'] = purchase_survey_long['TV Attributable Purchases'].fillna(0)


Lookup data was cleaned with the first two rows being dropped and the index was reset. The columns were renamed and any rows that have null values in the Exit survey column and airings network column were dropped.

In [ ]:
# Clean Lookup data
lookup_cleaned = lookup_df.drop([0, 1]).reset_index(drop=True)
lookup_cleaned.columns = ['Exit Survey', 'Airings Network', 'Survey Label']

This code creates a mapping between the 'Exit Survey' column and the 'Airings Network' column from the lookup_cleaned DataFrame. It uses this mapping to associate the 'Source' values in the purchase_survey_long DataFrame with the corresponding 'Network'. Additionally, the code cleans the airings_df DataFrame by converting the 'Date/Time ET' column to a standard date format, extracting only the date component and ensuring it's properly formatted for further analysis.

In [ ]:
# Create a mapping from 'Exit Survey' to 'Airings Network'
airing_map = dict(zip(lookup_cleaned['Exit Survey'], lookup_cleaned['Airings Network']))

# Apply mapping to purchase survey data
purchase_survey_long['Network'] = purchase_survey_long['Source'].map(airing_map)

# Clean airing data
airings_df['Date'] = pd.to_datetime(airings_df['Date/Time ET']).dt.date
airings_df['Date'] = pd.to_datetime(airings_df['Date'])


This code merges two datasets, airings_df and purchase_survey_long, based on the 'Date' and 'Network' columns, using an outer join to ensure no data is lost during the merge. It then flags specific dates (missing_dates) that are missing from the Purchase Survey by creating a new column called 'Missing_Purchase_Data'. Finally, it handles missing values in the 'TV Attributable Purchases' and 'Lift' columns by filling any NaN values with 0, ensuring that the data is clean and ready for further analysis.

In [ ]:
# Merge datasets
merged_df = pd.merge(airings_df, purchase_survey_long, on=['Date', 'Network'], how='outer')

# Create a flag for dates missing from Purchase Survey
missing_dates = [datetime(2017, 9, 1), datetime(2017, 9, 9), datetime(2017, 9, 20), datetime(2017, 10, 6)]
merged_df['Missing_Purchase_Data'] = merged_df['Date'].isin(missing_dates)

merged_df['Missing_Purchase_Data'] = merged_df['Date'].isin(missing_dates)

# Fill NaN values with 0 for 'TV Attributable Purchases' and 'Lift'
merged_df['TV Attributable Purchases'] = merged_df['TV Attributable Purchases'].fillna(0)
merged_df['Lift'] = merged_df['Lift'].fillna(0)


# Data Quality Analysis and Cleaning:

The following three assumptions were made:

1.   Lift value is additional traffic over the benchmark or normal traffic received thus it was assumed that the lift value had to be positive.

2.   The missing three days of customer purchase exit survey data any networks that advertised would have missleading CR & CPA.

3.   Analyzing advertising effectiveness where spend is zero or negative may be miss leading. Zero or negetive spend will be removed.

4.   For missing or inconsistent purchase/spend data, we would impute the average TV Attributable Purchases for the selected network.
-------
# Estimating Missing Purchases:
*   The function is designed to performs data quality checks cleaning on the DataFrame. It checks for missing values across the DataFrame and fills them with 0 while logging the affected columns in an issues list.

*   It then checks for negative values in the 'Lift' column, and while it logs a warning if any negative values are present, it does not modify these values, assuming they may still be valid for analysis.

*   Next, it verifies the 'Spend' column for negative values, and the corresponding rows are removed, and an error is recorded.

*   The function also checks for inconsistencies between 'Spend' and 'TV Attributable Purchases', identifying rows where there is spending but no purchases or vice versa, and logs this as a warning.

*   The function reviews the 'Missing_Purchase_Data' column to identify rows with missing purchase data, adding a warning for these rows.

*   Once the checks are complete, the function returns the cleaned DataFrame alongside a list of data quality issues for further review.

In [ ]:
# Function to estimate missing purchases
def estimate_missing_purchases(df):
    # Create a copy of the dataframe to avoid modifying the original
    df_copy = df.copy()

    # Calculate network-specific purchase-to-spend ratios, ensuring no division by zero
    network_ratios = df_copy.groupby('Network').apply(lambda x: (x['TV Attributable Purchases'].sum() / x['Spend'].sum()) if x['Spend'].sum() > 0 else 0)

    # Find rows where 'TV Attributable Purchases' is 0 and we need to estimate values
    zero_tv_attributable_index = df_copy[df_copy['TV Attributable Purchases'] == 0].index

    # Estimate 'TV Attributable Purchases' for the rows where it is 0 using the network ratio
    df_copy.loc[zero_tv_attributable_index, 'TV Attributable Purchases'] = df_copy.loc[zero_tv_attributable_index, 'Spend'] * df_copy.loc[zero_tv_attributable_index, 'Network'].map(network_ratios)

    return df_copy

    return df


# Data Quality Analysis and Cleaning
def check_and_clean_data(df):
    issues = []

    # Check for missing values
    missing_values = df.isnull().sum()
    if missing_values.sum() > 0:
        issues.append(f"Missing values found and filled:\n{missing_values[missing_values > 0]}")
        df = df.fillna(0)  # Fill missing values with 0

    # Check for negative Lift
    negative_lift_count = (df['Lift'] < 0).sum()
    if negative_lift_count > 0:
        issues.append(f"Warning: Found {negative_lift_count} instances of negative Lift")
        issues.append("Negative Lift values have been retained for analysis, but will be handled carefully in metric calculations.")

    # Check for zero or negative Spend
    nz_spend_count = (df['Spend'] <= 0).sum()
    if nz_spend_count > 0:
        issues.append(f"Error: Found and removed {nz_spend_count} instances of zero or negative Spend")
        issues.append("Assumption: There is no point analyzing advertising effectiveness where spend is zero or negative")
        issues.append("Warning: Removed rows where spend is zero or negative")
        df = df[df['Spend'] > 0]  # Remove rows with zero and negative Spend


    # Check for inconsistencies (e.g., spend with no purchases or vice versa)
    inconsistent = df[(df['Spend'] > 0) & (df['TV Attributable Purchases'] == 0) |
                      (df['Spend'] == 0) & (df['TV Attributable Purchases'] > 0)]
    if not inconsistent.empty:
        issues.append(f"Warning: Found {len(inconsistent)} rows with spend but no purchases or vice versa")
        estimate_missing_purchases(df)


    # Check for zero or negative Spend & TV Attributable Purchases
    nz_spend_count = ((df['Spend'] <= 0) | (df['TV Attributable Purchases'] <= 0)).sum()
    if nz_spend_count > 0:
        issues.append(f"Error: Found and removed {nz_spend_count} instances of zero or negative Spend")
        issues.append("Assumption: There is no point analyzing advertising effectiveness where spend and TV Attributable Purchases are both zero or negative")
        issues.append("Warning: Removed rows where spend and TV Attributable Purchases are both zero or negative")
        #df = df[df['Spend'] > 0]  # Remove rows with zero and negative Spend
        df = df[(df['Spend'] > 0) | (df['TV Attributable Purchases'] > 0)] # Remove rows with zero and negative Spend


    # Check for rows with missing purchase data
    missing_purchase_data_count = df['Missing_Purchase_Data'].sum()
    if missing_purchase_data_count > 0:
        issues.append(f"Warning: Found {missing_purchase_data_count} rows with missing purchase survey data")

    return df, issues

    # Check for unusually high values (potential outliers)
    for col in ['Spend', 'Lift', 'TV Attributable Purchases']:
        upper_bound = df[col].mean() + 3 * df[col].std()
        if (df[col] > upper_bound).any():
            issues.append(f"Potential outliers found in {col}")

    # Check for inconsistencies in date ranges
    date_range_exit = exit_survey_long['Date'].max() - exit_survey_long['Date'].min()
    date_range_airing = airing_df['Date'].max() - airing_df['Date'].min()
    if date_range_exit != date_range_airing:
        issues.append("Inconsistent date ranges between exit survey and airing data")

original_df = merged_df.copy()
# Apply data quality check and cleaning
merged_df, data_quality_issues = check_and_clean_data(merged_df)

<ipython-input-8-c149879d8f3f>:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  network_ratios = df_copy.groupby('Network').apply(lambda x: (x['TV Attributable Purchases'].sum() / x['Spend'].sum()) if x['Spend'].sum() > 0 else 0)


In [ ]:
d1 = original_df.sort_values('Lift', ascending=True)[original_df['Lift'] < (-30)]
d1[['Date','Network','Spend','Lift']]

<ipython-input-9-07f0a421c9cd>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  d1 = original_df.sort_values('Lift', ascending=True)[original_df['Lift'] < (-30)]


,Date,Network,Spend,Lift
5653,2017-10-12,WILO,17.19,-99.0
6869,2017-10-20,ZEETV,31.45,-62.0
821,2017-09-14,WILO,14.46,-57.0
7432,2017-10-25,WILO,16.20,-57.0
7435,2017-10-25,WILO,16.20,-53.0
6746,2017-10-20,OANN,60.69,-44.0
6808,2017-10-20,WILO,12.70,-42.0
7433,2017-10-25,WILO,15.56,-41.0
6866,2017-10-20,ZEETV,30.82,-40.0
5144,2017-10-09,STARPLUS,20.82,-40.0


# Performance Metrics

The calculate_metrics function computes key performance metrics from a given DataFrame, with the option to group the data by a specified column or by 'Network' if no group is provided. It aggregates several columns, such as 'Spend', 'Lift', 'TV Attributable Purchases', and 'Missing_Purchase_Data', for each group and calculates metrics like Cost Per Visitor (CPV), Conversion Rate (CR), and Cost Per Acquisition (CPA), ensuring that divisions by zero are handled by assigning inf (infinity) where necessary. It also computes the Return on Investment (ROI) by dividing 'TV Attributable Purchases' by 'Spend', and flags any groups that have a negative lift by adding a 'Has Negative Lift' column. The 'Missing_Purchase_Data' column is renamed for clarity, and any infinite values are replaced with NaN to facilitate better data visualization. Finally, the function returns the calculated metrics, rounding them to two decimal places for readability.


Cost Per Visitor (CPV): This metric is calculated by dividing the total spend by the lift (number of visitors driven by the campaign). It represents how much it costs to bring one visitor to the website or store. A lower CPV indicates better cost efficiency, as fewer dollars are spent per visitor.

Conversion Rate (CR): The CR is derived by dividing the number of TV Attributable Purchases by the lift (visitors). It reflects the percentage of visitors that actually make a purchase. A higher conversion rate shows the network's ability to not just attract visitors but also turn them into customers, which is a crucial measure of campaign effectiveness.

Cost Per Acquisition (CPA): CPA is calculated by dividing the total spend by the number of TV Attributable Purchases. This metric indicates the cost required to acquire one customer. A lower CPA is desirable, as it reflects the campaign’s ability to generate more customers for less money, which is key for profitability.


Has Negative Lift(Not currently being used but can be implimented due to not considering negetive lift values during calculations):

This flag indicates whether a particular network has a negative lift, meaning the campaign may have resulted in fewer visitors than expected. Negative lift is an anomaly that should be investigated as it could suggest a misalignment in the campaign's targeting or content.

Has Missing Purchase Data: This flag alerts if there were airings on the three days missing from the customer exit aurvey data.

In [ ]:
def calculate_metrics(df, group_by=None):
    if group_by:
        grouped = df.groupby(group_by)
    else:
        grouped = df.groupby('Network')

    metrics = grouped.agg({
        'Spend': 'sum',
        'Lift': 'sum',
        'TV Attributable Purchases': 'sum',
        'Missing_Purchase_Data': 'any'
    }).reset_index()

    # Handle potential division by zero or negative values
    metrics['CPV'] = metrics.apply(lambda row: row['Spend'] / row['Lift'] if row['Lift'] > 0 else np.inf, axis=1)
    metrics['CR'] = metrics.apply(lambda row: row['TV Attributable Purchases'] / row['Lift'] if row['Lift'] > 0 else 0, axis=1)
    metrics['CPA'] = metrics.apply(lambda row: row['Spend'] / row['TV Attributable Purchases'] if row['TV Attributable Purchases'] > 0 else np.inf, axis=1)


    # Add a 'Negative Lift' flag
    metrics['Has Negative Lift'] = metrics['Lift'] < 0

    # Rename the 'Missing_Purchase_Data' column for clarity
    metrics = metrics.rename(columns={'Missing_Purchase_Data': 'Has_Missing_Purchase_Data'})

    # Replace infinity with NaN for better visualization
    metrics = metrics.replace([np.inf, -np.inf], np.nan)

    #1 Replace infinity and NaN values with 0
    metrics = metrics.replace([float('inf'), float('-inf')], 0).fillna(0)

    return metrics.round(2)

In [ ]:
# Calculate metrics for different granularities
#original_metrics = calculate_metrics(original_df)
overall_metrics = calculate_metrics(merged_df)
monthly_metrics = calculate_metrics(merged_df, group_by=[merged_df['Date'].dt.to_period('M')])
network_monthly_metrics = calculate_metrics(merged_df, group_by=[merged_df['Date'].dt.to_period('M'), 'Network'])


In [ ]:
def generate_dq_report(original_df, merged_df, data_quality_issues):
    report = "Data Quality Analysis \n\n"

    report += f"Original number of rows: {len(original_df)}\n"
    report += f"Number of rows after cleaning: {len(merged_df)}\n"
    report += f"Rows removed in cleaning: {len(original_df) - len(merged_df)}\n\n"

    report += "Data Quality Issues:\n"
    for issue in data_quality_issues:
        report += f"- {issue}\n"

    return report

In [ ]:
def create_visualizations1(overall_metrics, monthly_metrics, merged_df):
    network_monthly_metrics['Date'] = network_monthly_metrics['Date'].astype(str)
    fig = px.line(network_monthly_metrics, x='Date', y='CPA', color='Network',
                  title='Cost Per Acquisition (CPA) Over Time by Network',
                  log_y=True)  # Set y-axis to logarithmic scale
    fig.update_layout(xaxis_tickangle=-45)
    fig.update_yaxes(title_text='CPA (log scale)')
    fig.show()


    fig = px.bar(network_monthly_metrics,
                x='Network',
                y='CPA',
                color='Date',
                title='Monthly CPA for All Networks',
                labels={'Network': 'Network', 'CPA': 'CPA ($)', 'Date': 'Month'},
                log_y=True,  # Using log scale due to wide range of values
                height=600)  # Increased height for better readability

    fig.update_layout(
        xaxis_tickangle=-45,
        barmode='group'
    )

    fig.show()


In [ ]:
# Call the function to create visualizations
create_visualizations1(overall_metrics, monthly_metrics, merged_df)

# Visualizations:

In [ ]:
# Generate the report
report = generate_dq_report(original_df, merged_df, data_quality_issues)

# Print the report
print(report)

def create_visualizations(overall_metrics, monthly_metrics, merged_df):
    # CPA by Network
    fig = px.bar(overall_metrics.sort_values('CPA', ascending=False),
                 x='Network', y='CPA',
                 title='Cost Per Acquisition (CPA) by Network')
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()

    network_monthly_metrics['Date'] = network_monthly_metrics['Date'].astype(str)
    fig = px.line(network_monthly_metrics, x='Date', y='CPA', color='Network',
                  title='Cost Per Acquisition (CPA) Over Time by Network',
                  log_y=True)  # Set y-axis to logarithmic scale
    fig.update_layout(xaxis_tickangle=-45)
    fig.update_yaxes(title_text='CPA (log scale)')
    fig.show()




    # Heatmap of purchase patterns
    pivot_df = merged_df.pivot_table(values='TV Attributable Purchases',
                                     index='Network', columns='Date',
                                     aggfunc='sum', fill_value=0)
    fig = px.imshow(pivot_df,
                    labels=dict(x="Date", y="Network", color="Purchases"),
                    color_continuous_scale='Rainbow',
                    title='Purchase Patterns by Network and Date')
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()

    # Lift Distribution by Network
    fig = px.box(merged_df, x='Network', y='Lift',
                 title='Lift Distribution by Network')
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()

    print("All visualizations have been displayed.")

# Call the function to create visualizations
create_visualizations(overall_metrics, monthly_metrics, merged_df)

Data Quality Analysis 

Original number of rows: 7917
Number of rows after cleaning: 6394
Rows removed in cleaning: 1523

Data Quality Issues:
- Missing values found and filled:
Company         1474
Date/Time ET    1474
Rotation        5899
Creative        1474
Network          336
Spend           1474
Program         2188
Source           682
dtype: int64
- Warning: Found 2403 instances of negative Lift
- Negative Lift values have been retained for analysis, but will be handled carefully in metric calculations.
- Error: Found and removed 1523 instances of zero or negative Spend
- Assumption: There is no point analyzing advertising effectiveness where spend is zero or negative
- Warning: Removed rows where spend is zero or negative
- Warning: Found 4110 rows with spend but no purchases or vice versa
- Error: Found and removed 4110 instances of zero or negative Spend
- Assumption: There is no point analyzing advertising effectiveness where spend and TV Attributable Purchases are both ze

All visualizations have been displayed.


# Data visualization and insights:

Lift Distribution by Network:
---
The box plot showing lift distribution requires careful interpretation due to the presence of apparently negative values, which are likely erroneous given that lift should not be negative.

Key insights:
---
* CNN demonstrates the highest median lift and widest range of positive values, indicating strong but variable performance.

* MSNB, HIST, and NFLN also show relatively high median lifts and larges range of Lift values indicating variability (inconsistent performance).

* Some networks (e.g., CNBCWORLD, DATELINE) have relatively compact distributions, suggesting more consistent Lift values.

* Data for networks showing mostly negative values (e.g., WILO, TWC, STARPLUS) needs to be rechecked and recalculated to understand their true performance.

The presence of these anomalies like variability & negative values indicates a need for data validation and possible adjustment of spend for beter targeting.

Purchase Patterns by Network and Date:
---
The heatmap reveals distinct patterns in purchase activity over time:

* WILO stands out with the most consistent and highest purchase activity throughout the period.
* Sporadic bursts of high purchase activity are observed for networks like DISH, DATELINE, OANN, STARPLUS and CNBCWORLD. DISH network shows a notable spike in purchases around September 24-26. OANN, STARPLUS, and CNBCWORLD show occasional spikes of activity but are mostly quiet.
* Many networks show little to no purchase activity for extended periods. There's a noticeable increase in overall activity around mid-September across several networks.

This pattern suggests varying effectiveness of marketing campaigns or content across networks and time periods.

Cost Per Acquisition (CPA) Over Time by Network:
--
The line graph tracking CPA trends shows:


* A general trend of gradually increasing CPA over time for most networks.
* HIST has the highest and most rapidly increasing CPA.
* Networks like DISH, COM, and TWC maintain relatively stable and low CPAs.
* Significant variation in CPA levels across networks, ranging from under 10 to nearly 1000.

This indicates differing efficiency in acquisition strategies among networks, with some maintaining cost-effectiveness over time while others become increasingly expensive.

Cost Per Acquisition (CPA) by Network:
---
The bar chart comparing CPAs across networks reveals:


* COM has the highest CPA by a significant margin, followed by SPECSPTS and HIST. These are the least Cost-Efficient Networks
* DISH, CNBCWORLD, and WILO have the lowest CPAs. These are the most Cost-Efficient Networks.
* A wide range of CPAs across networks, suggesting varying efficiency in acquisition strategies.

Overall Analysis:
---
**Performance Variability**: There's significant variation in performance across networks in terms of lift, purchase activity, and cost efficiency.

**High Performers**: CNN shows high lift and sporadic high purchase activity but with a moderate CPA. WILO demonstrates consistent purchase activity and low CPA, but its lift data needs verification.

**Cost Efficiency**: Networks like DISH and CNBCWORLD maintain low CPAs, making them cost-effective options, although their lift and purchase activity are not standout.

**Challenging Performers**: Networks like COM and SPECSPTS have very high CPAs, which don't seem justified by their lift or purchase activity, suggesting inefficient acquisition strategies.

**Temporal Trends**: The mid-October increase in purchase activity across several networks suggests a seasonal effect or successful cross-network campaign.

**Data Quality Concerns**: The lift data, in particular, needs careful review and possible recalculation to ensure accurate performance assessment.

Recommendations:
---

**Data Validation**: Conduct a thorough review of the lift calculation methodology and data processing to address the negative lift values issue.

**Focus on Efficiency**: Prioritize networks with high lift, consistent purchase activity, and low CPAs (e.g., DISH, potentially WILO after data verification).

**Optimize High-Cost Channels**: Investigate and potentially restructure strategies for high-CPA networks like COM and SPECSPTS.

**Temporal Strategy**: Capitalize on the mid-October activity surge by planning targeted campaigns during this period across multiple networks.

**Continuous Monitoring**: Implement ongoing performance tracking to quickly identify and respond to changes in lift, purchase patterns, and CPA across networks.

# Data Inconsitency Analysis:

# Report:

In [ ]:
def generate_report(original_df, cleaned_df, overall_metrics, monthly_metrics, network_monthly_metrics, data_quality_issues):
    report = "Data Quality Analysis Report\n\n"

    report += f"Original number of rows: {len(original_df)}\n"
    report += f"Number of rows after cleaning: {len(cleaned_df)}\n"
    report += f"Rows removed in cleaning: {len(original_df) - len(cleaned_df)}\n\n"

    report += "1. Overall Metrics:\n"
    report += overall_metrics.sort_values('CPA', ascending=False).to_string() + "\n\n"

    columns_to_check = ['Spend', 'TV Attributable Purchases']
    filtered_df = overall_metrics[(overall_metrics[columns_to_check]>0).all(axis=1) > 0]

    report += "2. Top 5 Most Cost-Efficient Networks:\n"
    report += filtered_df.nsmallest(5, 'CPA').to_string() + "\n\n"

    report += "3. Bottom 5 Least Cost-Efficient Networks:\n"
    report += overall_metrics.nlargest(5, 'CPA').to_string() + "\n\n"

    report += "4. Networks with Negative Lift:\n"
    negative_lift_networks = original_df[original_df['Lift'] < 0]['Network'].tolist()
    unique_negative_lift_networks = list(set(negative_lift_networks))
    report += ", ".join(unique_negative_lift_networks) + "\n\n"


    report += "5. Networks with Missing Purchase Data:\n"
    missing_data_networks = overall_metrics[overall_metrics['Has_Missing_Purchase_Data']]['Network'].tolist()
    report += ", ".join(missing_data_networks) + "\n"
    report += "These networks have airings on dates where purchase survey data is missing.\n\n"

    report += "6. Data Quality Issues:\n"
    for issue in data_quality_issues:
        report += f"- {issue}\n"

    report += "\n9. Visualization Insights:\n"
    report += "Interactive visualizations have been displayed using Plotly. They include:\n"
    report += "- Cost Per Acquisition (CPA) by Network\n"
    report += "- Cost Per Acquisition (CPA) Over Time by Network\n"
    report += "- Purchase Patterns by Network and Date\n"
    report += "- Lift Distribution by Network\n"
    report += "Please refer to these interactive plots for detailed visual insights.\n"

    best5 = filtered_df.nsmallest(5, 'CPA')
    worst5 = overall_metrics.nlargest(5, 'CPA')

    report += "\n10. Recommendations:\n"
    report += "- Investigate and potentially tweak the advertisements or reduce spend on sites with high CPA\n"
    report += "- Consider reducing ad spend on the least cost-efficient networks:\n\t"
    report += ", ".join(worst5['Network'].values.tolist()) + "\n"
    report += "- Consider increasing ad spend on the most cost-efficient networks:\n\t"
    report += ", ".join(best5['Network'].values.tolist()) + "\n"
    report += "- Negative Lift doesn't necessarily mean that the campaigns were a complete failure, but they didn't perform as well as expected.\n"
    report += "- Conduct in-depth analysis of networks with negative lift to understand underlying causes.\n"
    report += "- Address data quality issues, particularly for networks with inconsistent spend and purchase data.\n"
    report += "- Investigate the impact of missing purchase data on the affected networks.\n"
    report += "- Consider methods to estimate or account for missing purchase data in future analyses.\n"
    report += "- Implement a regular review process to monitor CPA and lift trends and quickly adjust strategies.\n"
    report += "- Consider multi-touch attribution models to better understand TV's impact alongside other channels.\n"

    report += "\nMonthly Metrics:\n"
    report += monthly_metrics.to_string() + "\n\n"

    report += "Network and Monthly Metrics:\n"
    #report += network_monthly_metrics.head(10).to_string() + "\n\n"
    report += network_monthly_metrics.sort_values('TV Attributable Purchases', ascending=False).to_string() + "\n\n"

    return report

In [ ]:
# Generate the report
report = generate_report(original_df, merged_df, overall_metrics, monthly_metrics, network_monthly_metrics, data_quality_issues)

# Print the report
print(report)

# Save the report to a file
with open('data_quality_report.txt', 'w') as f:
    f.write(report)

print("\nAnalysis complete. Report saved as 'data_quality_report.txt'.")
print("Interactive visualizations have been displayed.")

Data Quality Analysis Report

Original number of rows: 7917
Number of rows after cleaning: 6394
Rows removed in cleaning: 1523

1. Overall Metrics:
      Network     Spend    Lift  TV Attributable Purchases  Has_Missing_Purchase_Data    CPV    CR      CPA  Has Negative Lift
4         COM   7501.25   108.0                        4.0                      False  69.46  0.04  1875.31              False
15        TNT   5125.50   179.0                        3.0                      False  28.63  0.02  1708.50              False
13   SPECSPTS   5580.26    74.0                        4.0                      False  75.41  0.05  1395.06              False
8        HIST   7278.30   319.0                        7.0                      False  22.82  0.02  1039.76              False
10       NFLN   5412.80   200.0                        6.0                      False  27.06  0.03   902.13              False
12        SCI   7993.18   132.0                        9.0                      False  60.

# Final Data Quality Analysis:

Following the data cleaning and analysis, the report highlights key findings regarding the effectiveness and cost-efficiency of TV networks used for advertising.


Data Quality Improvements:

The original dataset had 7,917 rows, which were reduced to 6,394 after cleaning. 1,523 rows were removed due to issues such as zero or negative spend and other inconsistencies. Major missing values were identified in fields like Company, Date/Time ET, Network, Spend, and Program. While these missing values were filled, it remains important to monitor and address such gaps in future data collections.Negative Lift was found in 2,403 instances, indicating some campaigns performed worse than expected. These values were retained for careful analysis.

Top Networks for Cost Efficiency:

The most cost-effective networks include DISH, CNBCWORLD, WILO, STARPLUS, and DATELINE, with DISH standing out as the best performer with the lowest CPA (Cost Per Acquisition) of 1.25.
These networks delivered high purchase attribution and lift relative to the spend, suggesting that continuing or increasing investment in these networks could yield better returns.

Least Cost-Efficient Networks:

Networks like COM, SPECSPTS, HIST, NFLN, and CNN were the least cost-efficient, with significantly higher CPAs. COM had the highest CPA of 750.12, indicating inefficiency in driving purchases compared to spend.
It is recommended to review or reduce spend on these networks, or investigate why their performance is low despite high spending.

Networks with Airings on the days with Missing Purchase Data:

Networks BLOM, OANN, WILO, and ZEETV had significant portions of missing purchase data. The impact of this missing data should be carefully examined, and future data collection processes should aim to minimize such gaps.
Key Data Quality Issues:

Zero or negative spend values led to the removal of 1,523 rows. Further, 4,110 rows were identified where spend or purchases were missing, which introduces challenges in interpreting advertising effectiveness.
A total of 198 rows had missing purchase survey data, which may affect the accuracy of the analysis for certain networks.

Recommendations:
---
Increase spend on the most cost-efficient networks (DISH, CNBCWORLD, WILO, STARPLUS, DATELINE) to maximize returns on investment.
Reduce or re-evaluate spend on the least efficient networks (COM, SPECSPTS, HIST, NFLN, CNN) to avoid wasting budget on underperforming platforms.
Investigate the causes of negative lift to understand why certain campaigns underperformed.
Address data quality issues by improving processes for collecting purchase data and ensuring spend is accurately recorded across networks.
Consider the implementation of multi-touch attribution models to better understand the role of TV advertising alongside other channels in driving conversions.
A regular review process should be established to monitor trends in CPA and lift, allowing for timely adjustments to ad strategies.

---


# Findings for Monthly Metrics

Overall Spend and Performance:
---
For September 2017, the total spend amounted to 107,929.14, yielding a lift of 9,312 and 5,776 TV attributable purchases. The cost per view (CPV) was relatively low at 11.59, and the conversion rate (CR) was a strong 0.62. This indicates effective utilization of the advertising budget during this month.
In October 2017, the spend increased slightly to 113,507.70, with a corresponding lift of 10,929 and 7,005 purchases. The CPV improved to $10.39, and the CR remained consistent at 0.64, suggesting continued advertising effectiveness and improved conversion.


Trends Observed:
---
There is a positive trend in the lift and TV attributable purchases month-over-month, indicating that advertising efforts are yielding increasing returns over time. The maintained or slightly improved CPV suggests cost efficiency is being preserved even with increased spend.
The consistency in conversion rates across both months suggests that the target audience's response to advertisements remains favorable.
Findings for Network and Monthly Metrics

Network-Specific Insights:
---
WILO stands out in October with a significant spend of 27,961.08, yielding a lift of 5,000 and 3,779 purchases. Despite missing purchase data, the network's performance indicates that it is a significant player in the advertising landscape.
DISH had a lower spend of $2,513.09 in September, resulting in a commendable lift of 976 and a high CR of 2.06. This suggests that the network is highly effective in converting views to purchases with minimal expenditure.

Month-on-Month Comparison:
---
Comparing the two months, some networks displayed fluctuations in performance. For instance, while STARPLUS had a significant lift in October, it also had a higher CPA of 11.05 compared to its 6.83 in September, indicating a potential need for reevaluation of its advertising strategy.
ZEETV showed considerable variability with a spend of 15,429.12 in October but recorded missing purchase data, which could hinder a full evaluation of its performance.

Cost Efficiency Analysis:
---
The analysis highlights that certain networks, such as DISH and CNBCWORLD, consistently exhibit high CRs and low CPAs, indicating their effectiveness in converting advertising spend into actual purchases.
On the other hand, networks like COM and CNN are less cost-efficient, with high spend relative to their lift and purchases, suggesting a review of their advertising approaches may be beneficial.